In [1]:
import cv2
import numpy as np
import os

# 얼굴 탐지 모델 가중치
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascade_default_path = "xml/haarcascade_frontalface_default.xml"
cascade_alt = 'xml/haarcascade_frontalface_alt.xml'
faceCascade = cv2.CascadeClassifier(cascade_default_path);
font = cv2.FONT_HERSHEY_SIMPLEX
age_net = cv2.dnn.readNetFromCaffe(
                                    'weight/deploy_age.prototxt',
                                    'weight/age_net.caffemodel')

gender_net = cv2.dnn.readNetFromCaffe(
                                    'weight/deploy_gender.prototxt',
                                    'weight/gender_net.caffemodel')


# 모델 불러오기
cascade = cv2.CascadeClassifier(cascade_alt)


#인덱스 몇부터 시작할지
id = 0

# names related to ids: example ==> None: id=0,  etc
# 이런식으로 사용자의 이름을 사용자 수만큼 추가해준다.
names = ['None', 'OIJ', 'SYS', 'JSJ', 'JYS', 'ORK']

age_list = ['(0 ~ 2)','(4 ~ 6)','(8 ~ 12)','(15 ~ 20)',
            '(25 ~ 32)','(38 ~ 43)','(48 ~ 53)','(60 ~ 100)']
gender_list = ['Male', 'Female']

#default values
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)


# 이니셜라이즈
cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)

# 윈도우 사이즈
#minW = 0.1*cam.get(3)
#minH = 0.1*cam.get(4)


# 영상 파일 
video = cv2.VideoCapture('sample/mp4/sample1.mp4')
# 이미지 파일
img = cv2.imread('sample/jpg/sample99.jpg')


# 사진 검출기
def imgDetector(img,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list):
    
    # 영상 압축
    img = cv2.resize(img,dsize=None,fx=1.0,fy=1.0)
    # 그레이 스케일 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    # cascade 얼굴 탐지 알고리즘 
    results = cascade.detectMultiScale(gray,            # 입력 이미지
                                       scaleFactor= 1.5,# 이미지 피라미드 스케일 factor
                                       minNeighbors=5,  # 인접 객체 최소 거리 픽셀
                                       minSize=(20,20)  # 탐지 객체 최소 크기
                                       )        

    
    
    for(x,y,w,h) in results:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # 확률 계산 ==> "0" is perfect match
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))

        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1) 
    
    
    
    for box in results:

        x, y, w, h = box
        face = img[int(y):int(y+h),int(x):int(x+h)].copy()
        blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        
        # gender detection
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_preds.argmax()
        
        # Predict age
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = age_preds.argmax()
        info = gender_list[gender] +' '+ age_list[age]
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,255), thickness=2)
        cv2.putText(img,info,(x,y-15),0, 0.5, (0, 255, 0), 1)

    # 사진 출력
    cv2.imshow('facenet',img)  
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
#영상 검출기
def videoDetector(video,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list):

    while True:

        # 캡처 이미지 불러오기
        ret,img = video.read()
        #img = cv2.flip(img, -1) # 카메라 반전
        # 그레이 스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        # 영상 압축
        try:
            img = cv2.resize(img,dsize=None,fx=1.0,fy=1.0)
        except:
            break

        # cascade 얼굴 탐지 알고리즘 
        results = cascade.detectMultiScale(gray,            # 입력 이미지
                                           scaleFactor= 1.1,# 이미지 피라미드 스케일 factor
                                           minNeighbors=5,  # 인접 객체 최소 거리 픽셀
                                           minSize=(20,20)  # 탐지 객체 최소 크기
                                           )

        
        for(x,y,w,h) in results:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

            # 확률 계산 ==> "0" is perfect match
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1) 
        
        for box in results:
            x, y, w, h = box
            face = img[int(y):int(y+h),int(x):int(x+h)].copy()
            blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            # gender detection
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = gender_preds.argmax()
            
            # Predict age
            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = age_preds.argmax()
            
            info = gender_list[gender] +' '+ age_list[age]

            cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,255), thickness=2)
            cv2.putText(img,info,(x,y-15),0, 0.5, (0, 255, 0), 1)


         # 영상 출력
        cv2.imshow('facenet',img)

        if cv2.waitKey(1) > 0: 
            cv2.destroyAllWindows()
            break

            
def camDetector(cam,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list):
    
    while True:

        # 캡처 이미지 불러오기
        ret,img = cam.read()
        #img = cv2.flip(img, -1) # 카메라 반전
        # 그레이 스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        
        # 영상 압축
        try:
            img = cv2.resize(img,dsize=None,fx=1.0,fy=1.0)
        except:
            break
            

        # cascade 얼굴 탐지 알고리즘 
        results = cascade.detectMultiScale(gray,            # 입력 이미지
                                           scaleFactor= 1.1,# 이미지 피라미드 스케일 factor
                                           minNeighbors=5,  # 인접 객체 최소 거리 픽셀
                                           minSize=(20,20)  # 탐지 객체 최소 크기
                                           )

        for(x,y,w,h) in results:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

            # 확률 계산 ==> "0" is perfect match
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1) 
        
        
        for box in results:
            x, y, w, h = box
            face = img[int(y):int(y+h),int(x):int(x+h)].copy()
            blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            # gender detection
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = gender_preds.argmax()
            
            # Predict age
            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = age_preds.argmax()
            
            info = gender_list[gender] +' '+ age_list[age]

            cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,255), thickness=2)
            cv2.putText(img,info,(x,y-15),0, 0.5, (0, 255, 0), 1)


         # 영상 출력
        cv2.imshow('facenet',img)

        if cv2.waitKey(1) > 0: 
            cv2.destroyAllWindows()
            break            


            
# 영상 탐지기
#videoDetector(video,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list )
# 사진 탐지기
#imgDetector(img,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list )
# 캠 탐지기
camDetector(cam,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list )
            
        
# 클린업
print("\n [INFO] 프로그램 종료")
cam.release()
cv2.destroyAllWindows()



 [INFO] 프로그램 종료


In [ ]:

#영상 검출기
def videoDetector(video,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list):

    while True:

        # 캡처 이미지 불러오기
        ret,img = video.read()
        #img = cv2.flip(img, -1) # 카메라 반전
        # 그레이 스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        # 영상 압축
        try:
            img = cv2.resize(img,dsize=None,fx=1.0,fy=1.0)
        except:
            break

        # cascade 얼굴 탐지 알고리즘 
        results = cascade.detectMultiScale(gray,            # 입력 이미지
                                           scaleFactor= 1.1,# 이미지 피라미드 스케일 factor
                                           minNeighbors=5,  # 인접 객체 최소 거리 픽셀
                                           minSize=(20,20)  # 탐지 객체 최소 크기
                                           )

        
        for(x,y,w,h) in results:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

            # 확률 계산 ==> "0" is perfect match
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1) 
        
        for box in results:
            x, y, w, h = box
            face = img[int(y):int(y+h),int(x):int(x+h)].copy()
            blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            # gender detection
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = gender_preds.argmax()
            
            # Predict age
            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = age_preds.argmax()
            
            info = gender_list[gender] +' '+ age_list[age]

            cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,255), thickness=2)
            cv2.putText(img,info,(x,y-15),0, 0.5, (0, 255, 0), 1)


         # 영상 출력
        cv2.imshow('facenet',img)

        if cv2.waitKey(1) > 0: 
            cv2.destroyAllWindows()
            break

        
# 캠 탐지기
camDetector(cam,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list )
            
        
# 클린업
print("\n [INFO] 프로그램 종료")
cam.release()
cv2.destroyAllWindows()
